# Solutions for Exercise 3 - Range proofs with ZKPs

This only holds the answer to the questions, and the solution to the last coding exercise.

---

## 1 - Challenges for Issuer - Setting up and signing a credential

In [ ]:
// Defining the schema and one credential.
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        // CHALLENGE 1.3
        placeOfOrigin: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0},
        postalCode: {type: 'integer', minimum: 1000, maximum: 9999},
        height: {type: 'integer', minimum: 130, maximum: 210},
        weight: {type: 'number', minimum: 40, multipleOf: 0.1}
    }
}

const E_ID_DATA = {
    name: "Jack Sparrow",
    // CHALLENGE 1.3
    placeOfOrigin: "Vierwaldstättersee",
    timeOfBirth: new Date("1993-08-01T00:00:00").getTime(),
    postalCode: 1000,
    profession: "IT Manager",
    height: 176,
    // CHALLENGE 1.1: we had a good meal, so increase the weight.
    weight: 90
}

In [ ]:
import { initializeWasm, BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES,
        CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR,
       KBUniversalAccumulator, dockAccumulatorParams, STATUS_STR, REV_ID_STR,
       NON_MEM_CHECK_STR, RevocationStatusProtocol } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

// Generating a keypair for the issuer.
// We suppose that there is a PKI available which lets the verifier
// know the public key of the issuer.
const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));
const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

// Issuing a credential based on the E-ID data based on the specified schema,
// and sign it using the BBS+ library.
const baseSchema = CredentialSchema.essential();
baseSchema.properties[SUBJECT_STR] = E_ID_SCHEMA;
// CHALLENGE 6 - add the status (revoked or not) to the schema
baseSchema.properties[STATUS_STR] = CredentialSchema.statusAsJsonSchema();
const eIDSchema = new CredentialSchema(baseSchema);

// CHALLENGE 6 - Setting up an accumulator and a witness of non-inclusion
const accumulatorKP = KBUniversalAccumulator.generateKeypair(dockAccumulatorParams());
const accumulatorSk = accumulatorKP.secretKey;
const accumulatorPk = accumulatorKP.publicKey;
const users = ['user1', 'user2'];
const users_encoded = users.map((user) => eIDSchema.encoder.encodeMessage(`${STATUS_STR}.${REV_ID_STR}`, user));
const accumulator = await KBUniversalAccumulator.initialize(users_encoded, dockAccumulatorParams(), accumulatorSk);

// CHALLENGE 6 - Creating the witness that this user hasn't been added to the accumulator.
const nonMemWitness = await accumulator.nonMembershipWitness(users_encoded[0], accumulatorSk);
accumulator.add(users_encoded[0], accumulatorSk);

const builder = new BBSCredentialBuilder();
builder.schema = eIDSchema;
builder.subject = E_ID_DATA;
// CHALLENGE 6 - Add the encoded userID to the credential.
builder.setCredentialStatus('dock:accumulator:accumId123', NON_MEM_CHECK_STR, users[0], RevocationStatusProtocol.KbUni24);
const credential = builder.sign(secretKey);

// This is the data which is sent to the holder.
const credentialToSendToHolder = credential.toJSON();
console.log("This data is sent to the holder:", credentialToSendToHolder);

### Challenges

1. Change the content of the credential

> Done in the creation of the credential.

2. Why is the `timeOfBirth` not a string like "21st of March 1998"?

> It is much more lightweight for a ZKP to prove that a positive number
> is in a given range than to create a proof for a string.
> An intuition is that for a number, the ZKP can directly prove that
> a certain bit-pattern is given.
> Whereas proving that a string represents a date within a given range
> requires much more description.

3. Add a new field to the credential, but don't forget to also add it to the schema

> Done in the creation of the credential.

---

## 2 - Challenges for Creating a range proof using LegoGroth16

In [ ]:
// The verifier is responsible for creating these keys, then sharing only the proving key with 
// the credential holder.
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

In [ ]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

// The holder creates a proof for their height, as requested from the verifier.
// Instead of signing it, they call `finalize` to create the proof.
// The use-case is left as an exercise to the reader.
// My oldest daughter once convinced my youngest daughter that there is a minimal height
// for admission to UNIL, to avoid people signing up their dogs.
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));

// CHALLENGE 2.1
// This will fail, as the holder cannot create that proof.
// builder.enforceBounds(0, 'credentialSubject.height', 70, 90, 'heightRangeCheck', snarkProvingKey);
// Create a proof that `170 <= height < 190`
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', snarkProvingKey);

// CHALLENGE 2.3 and 2.4
// Create a proof that `80 <= weight < 100`
builder.enforceBounds(0, 'credentialSubject.weight', 80, 100, 'weightRangeCheck', snarkProvingKey);

// CHALLENGE 5.1
// Create a proof that the holder is above 18.
// Calculate the latest birthday allowed for the holder to be at least
// 18 years old.
const currentDate = new Date();
const nowMinus18Years = new Date(currentDate.setFullYear(currentDate.getFullYear())).getTime();
builder.enforceBounds(0, 'credentialSubject.timeOfBirth', 0, nowMinus18Years, 'ageCheck', snarkProvingKey);
// CHALLENGE 6 - Add the proof of non-inclusion in the accumulator.
builder.addAccumInfoForCredStatus(0, nonMemWitness, accumulator.accumulated, accumulatorPk);
const presentation = builder.finalize();

// This is a serialized version, to be sent across an API to the verifier.
const lgProofForVerifier = presentation.toJSON();
console.log("Data sent to verifier:", lgProofForVerifier);

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

// Now the verifier can check whether the proof is valid.
// CHALLENGE 2.3, 2.4, and 5.1
// Add the additional / modified range check here, using the same verifying key.
const predicateParams = new Map([['heightRangeCheck', snarkVerifyingKey],
                                ['weightRangeCheck', snarkVerifyingKey],
                                ['ageCheck', snarkVerifyingKey]]);

// CHALLENGE 6 - Hand over the public key of the accumulator to the verify method.
// Here we don't check for the up-to-date accumulator.
const accParams = new Map([[0, accumulatorPk]]);
const recreatedPres = Presentation.fromJSON(lgProofForVerifier)
console.log("Verification succeeded:", recreatedPres.verify([publicKey], accParams, predicateParams).verified);
console.log("Revealed attributes:");
console.dir(recreatedPres.spec.credentials[0].revealedAttributes, {depth: null});
console.log("Bounds proven:");
console.dir(recreatedPres.spec.credentials[0].bounds, {depth: null});
console.log("Revocation check:", recreatedPres.spec.credentials[0].status.revocationCheck);

### Challenges

1. What happens if you try to create a proof which is not satisfied by the credential? Why?

> The proof creation will fail, as it is not possible to create a mathematical represention
> of a wrong proof.
> 
> For most of the SNARKs the creator of the setup could use their `verifierKey` to create
> any proof.
> This is the reason that the setup phase is critical, and that the machine doing the setup
> is trusted.
> If it's only to prove something from the holder to the verifier, this is less of a problem,
> as the verifier is not interested to cheat on the proof it receives.

2. How does the verifier know the bounds check performed? Does it need this information? Why (not)?

> In a normal interaction, the verifier will send the bounds check to be performed to
> the holder.
> The library will include these bound checks in the proof being sent to the verifier.
> First of all the verifier needs to verify that the bound checks are what it wants
> them to be.
>
> Next the verifier needs to create the `statement x` which needs the bound checks.
> Once the `statement x` is created, the verifier can call it using the proof of the
> holder.
> If the proof is correct, the `statement x` will run successfully.

3. Change the proof to be on a different field of the credential
4. Add a second range proof

> Both are implemented using a second range proof on the weight.

---

## 3 - Challenges for Creating a range proof using Bulletproofs++

In [ ]:
import { BoundCheckBppParams, PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

// The holder creates a proof for their height, as requested from the verifier.
// Instead of signing it, they call `finalize` to create the proof.
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey);
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
const boundCheckBppParams1 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', boundCheckBppParams1);

// CHALLENGE 5.2
builder.enforceBounds(0, 'credentialSubject.postalCode', 1000, 1199, 'VaudCheck', boundCheckBppParams1);

// CHALLENGE 6 - Add the proof of absence in the accumulator to the proof.
builder.addAccumInfoForCredStatus(0, nonMemWitness, accumulator.accumulated, accumulatorPk);
const presentation = builder.finalize();

// This is a serialized version, to be sent across an API to the verifier.
const bpProofForVerifier = presentation.toJSON()
console.log("Data sent to the verifier:");
console.dir(bpProofForVerifier, {depth: null});

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

// The verifier now proceeds to check the proof to see if it's valid.
const boundCheckBppParams2 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();
const predicateParams1 = new Map([['heightRangeCheck', boundCheckBppParams2],
                                 ['VaudCheck', boundCheckBppParams2]]);

// CHALLENGE 6 - Hand over the public key of the accumulator to the verify method.
// Here we don't check for the up-to-date accumulator.
const accParams = new Map([[0, accumulatorPk]]);
const recreatedPres = Presentation.fromJSON(bpProofForVerifier);
console.log("Verification succeeded:", recreatedPres.verify([publicKey], accParams, predicateParams1).verified);

console.log("Revealed attributes:");
console.dir(recreatedPres.spec.credentials[0].revealedAttributes, {depth: null});
console.log("Bounds proven:");
console.dir(recreatedPres.spec.credentials[0].bounds, {depth: null});
console.log("Revocation check:", recreatedPres.spec.credentials[0].status.revocationCheck);

### Challenges

1. If you do the same exercises as for the LegoGroth16, what is the difference in the code?

> - The setup needs to be done both by the prover (holder) and the verifier.
>   As the setup is very fast, this is not a problem.
>   Also, the setup needs a common secret key, which can be the challenge sent by
>   the verifier to the prover (holder).
> - Besides that the code is the same, which shows the nice abstraction level
>   attained by the code.

2. What other difference do you experience when running the code?

> - Bulletproofs++ are much faster than LegoGroth16, because they are a much more
>   recent construct.
> - Bulletproofs++ create a bit larger proofs, but still in the low kBytes range.

3. Can you find how the verifier can be sure what is proven by the holder?

> Like for LegoGroth16, the bounds are in the structure sent to the verifier.
> Again, the verifier will have to re-create the `statement x`, before being
> able to plug-in the `proof p` and checking for a correct result.

---

## 4 - Discussion: are we anonymous and unlinkable now?

1. What did we gain by using Zero Knowledge Proofs?

> Taking up the "k-anonymity" from before (where "k" is the number of holders
> with the same attribute-value), we did increase "k" by a lot:
> there are much more people having a height between 170cm and 190cm,
> than people with a specific height like 176.
>
> Also, if we combine proofs, the "k" is still very hihg.
> Except if we reveal low-k values like the name or the profession.

2. What does the verifier learn about our credential?

> - That it is a signed credential from the issuer
> - The values we reveal
> - The `statement`s we prove

3. Does this make the proof anonymous? Why? Why not?

> Depending on the "k-anonymity" we can consider ourselves anonymous if "k" is
> above a certain threshold.
> If "k" drops below a threshold, we can consider that the holders can be
> de-anonymized.

4. Is the proof unlinkable? When is it? When isn't it?

> Same as for anonymity, because all the rest of the proof is randomized
> and different for each proof.

---

## 5 - Coding challenge - Using the ZKPs

Here are some more coding exercises to help you understand the code.
We propose you to copy the exercise into a new notebook, so you always have the original code ready for comparisons.

1. Create a proof that your age is in a certain range. Why is the age format given like this?

> This challenge has been added to the LegoGroth part of this exercise.
   
3. Add a new field for the postal code to the credential and to the schema. Now create a proof that you live in the canton de Vaud.

> As Ahmed shared during the hands-on workshop, the postal code in Switzerland are actually
> not organized by canton, but by regions.
> Unfortunately this doesn't allow us to simply prove that somebody lives in the canton
> de Vaud using their postal code.
> So we either need to have a field with the name of the canton, and reveal that one.
> Or create many range proofs and `OR` them together.

---

## 6 - Hard Coding Exercise - Revocation test: add an accumlator using a negative membership check

Implement a revocation test of the credential using an accumulator.

This exercise is very hard, as unfortunately the docknetwork/crypto library is lacking a good documentation.
A good starting point is the following test:
[presentation-status-kb-accumulator.spec.ts](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/tests/anonymous-credentials/presentation-status-kb-accumulator.spec.ts#L143).
If you want to dive into this, please keep in mind:

- The tests in the `crypto-wasm-ts` library are written to work with different proving systems in mind.
  So they have a lots of variables which are filled out by other parts of the tests to make it as universal as possible.
- My preferred way to follow around the code is using VisualStudio, and then use `<CMD+CLICK>` to follow methods, `<CMD+OPTION+LEFT>`
  to go back

An accumulator is a cryptographic tool which allows to do (negative) membership proofs.
Instead of simply listing a set of numbers, an accumulator compresses these numbers using cryptographic algorithms.
Different accumulators exist with different trade-ofs:

- the size of the accumulator, depending on the number of elements in it
- whether it's possible to add and remove elements once the accumulator is set up
- if you can prove the existence and/or the absence of an element in the accumulator

Using the docknetwork/crypto library, you can add an inclusion or an absence proof using accumulators.
In order to prove the validity of the credential, the following needs to be done:

1. The issuer holds the accumulator which contains all revoked elements
2. If a credential is revoked, its ID needs to be stored in the accumulator
3. The holder creates an additional proof that their ID is not in the accumulator
4. When the verifier checks the proof, it can be sure that the credential of the holder hasn't been revoked yet

From a programming point of view, you have to add the following:

- Issuer
  - Setup the accumulator using something like [setupKBUniAccumulator](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/tests/anonymous-credentials/utils.ts#L824)
    - the `state` is not necessary
    - instead of adding 100 IDs, start with adding only two possible IDs
  - use `builder.setCredentialStatus` with a non-member check, like [here](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/tests/anonymous-credentials/presentation-status-kb-accumulator.spec.ts#L97)
    - the `registryID` can be any string
    - the `memberValue` needs to be the encoded user-ID from the `schema.encoder.encodeMessage` call
    - the `extra` field with the `blockNo` is optional and can be ignored.
      It is used to transfer additional information from the holder to the verifier.
- Holder
  - call `builder.addAccumInfoForCredStatus` with the values from the issuer. These values are public.
- Verifier
  - add a map with the public key of the accumulator to the call of `recreatedPres.verify`

### Questions:

1. What security did we imply with this setup?

> The issuer has to be trusted: with the private key, the issuer can
> prove any value to be included or excluded in the accumulator!
> In case of revocation, the security is only that the issuer will not send an
> updated `witness` to the holder.
> Which means that the holder will only be able to prove that they were not 
> revoked in a previous accumulator.
   
2. What can go wrong?

> - The holder could use a stale accumulator proof when proving the statement.
> - As the issuer needs to send out updates of the accumulator to all holders,
>   this is usually only done on a periodic time base.
>   Between two updates, a holder might have a revoked credential which is still
>   valid until the next update.

3. How can the verifier be sure that it didn't get a stale proof of the accumulator?

> Like for any other of the proofs, the verifier can require the inclusion of a
> challenge by the prover.
> It can require this to be done directly in the `nonMemWitness` proof, or in
> the `nonce` as seen previously.

4. How can the verifier be sure that the proof hasn't been done on a stale accumulator?

> As the accumulator is part of the computation whether the `statement x` is correct
> for a given `proof p`, the verifier can check whether the accumulator sent by the
> holder is the latest one or not.